# Preprocessing the Phoneme Sequences
Here I have to process the phoneme sequences so that they can be fed into a Keras embedding layer

In [30]:
#preprocessing for using whole-sequence embedding approach
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import csv
import functools
import pandas as pd
import random
import blist

random.seed(285)

# need to read in csv file with pairs and labels
train = pd.read_csv("capstone_train_and_test/new_train.csv")
vocabulary_size = 39 #aka number of different phonemes
max_len = 6 #maximum size of a phoneme sequence. Average sequence is 6.34 phonemes
tokenizer = Tokenizer(num_words = vocabulary_size)
tokenizer.fit_on_texts(train["phonemic_transcriptions_1"]) #finds number of tokens (phonemes in this case)
train_sequences_1 = tokenizer.texts_to_sequences(train["phonemic_transcriptions_1"]) #translates all words to lists of integers
train_sequences_2 = tokenizer.texts_to_sequences(train["phonemic_transcriptions_2"])
train_data_1_concrete = pad_sequences(train_sequences_1, maxlen = max_len, padding = "post")
train_data_2_concrete = pad_sequences(train_sequences_2, maxlen = max_len, padding = "post")
print(train_data_1_concrete[10])

[ 5  6  7 20  6  0]


In [31]:
raw_labels = train["rhyme_percentile"]*10
categorized_labels_concrete = to_categorical(raw_labels) # the `y` label we're trying to fit to
train_data_1_concrete = list(train_data_1_concrete)
train_data_2_concrete = list(train_data_2_concrete)
categorized_labels_concrete = list(categorized_labels_concrete)

In [32]:
#find info about lists


In [33]:
train_data_1 = train_data_1_concrete
train_data_2 = train_data_2_concrete
categorized_labels = categorized_labels_concrete

def reduce_data():
    global train_data_1
    global train_data_2
    global raw_labels
    global categorized_labels
    
    #get number of words in each category
    print("Reduce Data process starting")
    category_amounts = [0 for i in range(11)]
    for i in range(len(raw_labels)):
        category_amounts[int(raw_labels[i])] += 1
    for i in range(len(category_amounts)):
        print(category_amounts[i])
    
    #go through each sample and remove based on fraction
    ceiling = 50000
    print("Marking for deletion starting")
    for i in range(len(raw_labels)):
        random_num = random.random()
        fraction = 1/(category_amounts[int(raw_labels[i])] / ceiling)
        if random_num >= fraction:
            # delete it
            train_data_1[i] = ""
            train_data_2[i] = ""
            categorized_labels[i] = [0.]
    print("Marking for deletion finished")
    print("Making filtered list starting")
    train_data_1 = [t for t in train_data_1 if t != ""]
    train_data_2 = [t for t in train_data_2 if t != ""]
    categorized_labels = [t for t in categorized_labels if sum(t) != 0.]
    print("Making filtered list finished")
    print("Reduce Data process finished")

reduce_data()
category_amounts = [0 for i in range(11)]
for i in range(len(categorized_labels)):
    current_label = categorized_labels[i]
    index = 0
    for u in range(len(current_label)):
        if current_label[u] == 1:
            index = u
    category_amounts[index] += 1
for i in range(len(category_amounts)):
    print(category_amounts[i])

Reduce Data process starting
192127
1117331
2289063
2279152
1159709
356762
84146
16621
3401
539
2148
Marking for deletion starting
Marking for deletion finished
Making filtered list starting


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Making filtered list finished
Reduce Data process finished
50182
50368
49506
49679
50030
50081
49962
16621
3401
539
2148


In [28]:
# build whole-sequence model
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Subtract, Conv1D

batch_size = 100
output_dim_size = 20

input_1 = Input(batch_shape=(batch_size, 17))
input_2 = Input(batch_shape=(batch_size, 17))
embedding = Embedding(vocabulary_size, output_dim_size, input_length=max_len)
conv1d_1 = Conv1D(100, 3, activation = "relu", strides=2)
conv1d_2 = Conv1D(50, 3, activation = "relu", strides=2)
conv1d_3 = Conv1D(25, 3, activation = "relu", strides=2)

embedding_1 = embedding(input_1)
embedding_2 = embedding(input_2)
conv1d_1_1 = conv1d_1(embedding_1)
conv1d_1_2 = conv1d_1(embedding_2)
conv1d_2_1 = conv1d_2(conv1d_1_1)
conv1d_2_2 = conv1d_2(conv1d_1_2)
conv1d_3_1 = conv1d_3(conv1d_2_1)
conv1d_3_2 = conv1d_3(conv1d_2_2)
merge_layer = Subtract()([conv1d_3_1, conv1d_3_2])
flatten = Flatten()(merge_layer)
dense_output_layer = Dense(11, activation="softmax", input_shape=(max_len*output_dim_size,))(flatten)

whole_sequence_model = Model([input_1, input_2], dense_output_layer)
whole_sequence_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           (100, 17)            0                                            
__________________________________________________________________________________________________
input_30 (InputLayer)           (100, 17)            0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (100, 17, 20)        780         input_29[0][0]                   
                                                                 input_30[0][0]                   
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (100, 8, 100)        6100        embedding_7[0][0]                
          

In [29]:
# Compile and fit model
print(len(train_data_1))
whole_sequence_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
whole_sequence_model.fit([np.array(train_data_1[:372500]), np.array(train_data_2[:372500])], 
                         np.array(categorized_labels[:372500]), 
                         epochs=10, batch_size=100, validation_split = 0.2)

372517
Train on 298000 samples, validate on 74500 samples
Epoch 1/10
298000/298000 [==============================] - 32s 109us/step - loss: 2.1245 - acc: 0.1829 - val_loss: 2.0456 - val_acc: 0.2133
Epoch 2/10
298000/298000 [==============================] - 29s 98us/step - loss: 2.0253 - acc: 0.2222 - val_loss: 2.0155 - val_acc: 0.2303
Epoch 3/10
298000/298000 [==============================] - 28s 94us/step - loss: 2.0059 - acc: 0.2317 - val_loss: 2.0051 - val_acc: 0.2311
Epoch 4/10
298000/298000 [==============================] - 30s 101us/step - loss: 1.9940 - acc: 0.2374 - val_loss: 1.9995 - val_acc: 0.2415
Epoch 5/10
298000/298000 [==============================] - 30s 100us/step - loss: 1.9864 - acc: 0.2400 - val_loss: 1.9904 - val_acc: 0.2403
Epoch 6/10
298000/298000 [==============================] - 33s 111us/step - loss: 1.9822 - acc: 0.2413 - val_loss: 1.9895 - val_acc: 0.2427
Epoch 7/10
298000/298000 [==============================] - 33s 112us/step - loss: 1.9788 - acc: 0

KeyboardInterrupt: 

In [11]:
#preprocessing if using phoneme embedding approach
# preprocessing to make the actual phoneme embedding

#tokenization
words = []
with open("transcriptions_data.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        words.append(row)
words = np.array(words).flatten()

vocabulary_size = 39 #aka number of different phonemes
max_len = 17 #maximum size of a phoneme sequence
tokenizer = Tokenizer(num_words = vocabulary_size)
tokenizer.fit_on_texts(words) #finds number of tokens (phonemes in this case)
sequences = tokenizer.texts_to_sequences(words) #translates all words to lists of integers
print(sequences[10])
data = pad_sequences(sequences, maxlen = max_len, padding = "post")
print(data[10])

#okay, so I have my sequences preprocessed
# wait, actually I need to read in each column of all the phoneme sequences matched up together and their categorizations and make the categorizations Keras-compliant

7500999
[24, 14, 2, 10, 19, 11]
[24 14  2 10 19 11  0  0  0  0  0  0  0  0  0  0  0]


In [24]:
#build phoneme embedding

In [ ]:
#use phoneme embedding in new model